In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Amyotrophic_Lateral_Sclerosis"
cohort = "GSE139384"

# Input paths
in_trait_dir = "../../input/GEO/Amyotrophic_Lateral_Sclerosis"
in_cohort_dir = "../../input/GEO/Amyotrophic_Lateral_Sclerosis/GSE139384"

# Output paths
out_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/GSE139384.csv"
out_gene_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/gene_data/GSE139384.csv"
out_clinical_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/clinical_data/GSE139384.csv"
json_path = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Synaptopathy in Kii ALS/PDC, a disease concept based on transcriptome analyses of human brains"
!Series_summary	"Amyotrophic lateral sclerosis (ALS) and parkinsonism-dementia complex (PDC) (ALS/PDC) is a unique endemic neurodegenerative disease, with high-incidence foci in the Kii Peninsula, Japan. Although ALS/PDC presents with multiple proteinopathies, the genetic and environmental factors that influence disease onset remain unknown. We performed transcriptome analyses of patients’ brains, which may provide new insights into the pathomechanisms underlying Kii ALS/PDC."
!Series_summary	"We prepared frozen brains from 3 healthy controls (frontal lobe and temporal lobe), 3 patients with Alzheimer’s disease (AD) (frontal lobe and temporal lobe) as tauopathy-disease controls, and 21 patients with Kii ALS/PDC (frontal lobe and/or temporal lobe). We acquired microarray data from the cerebral gray and white matter tissues of Kii ALS/PDC patients."
!Seri

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this dataset appears to contain gene expression data
# The study mentions "microarray data" and using HumanHT-12 v4 Expression BeadChip
# This indicates gene expression data, not just miRNA or methylation data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking for trait data (ALS) in the sample characteristics
# In key 0, we find 'clinical phenotypes: ALS', 'clinical phenotypes: ALS+D', etc.
# Key 1 contains 'clinical phenotypes: Healthy Control' and 'clinical phenotypes: Alzheimer`s Disease'
# This suggests key 0 contains the phenotype information for the subjects including ALS
trait_row = 0

# Looking for age data
# In key 2 and 3, we find multiple 'age: XX' entries
# This suggests age data is available
age_row = 2  # Choose key 2 as it has more age entries

# Looking for gender data
# In key 1, we find 'gender: Female' and 'gender: Male'
# In key 2, we find 'gender: Male'
# This suggests key 1 contains more complete gender information
gender_row = 1

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait value to binary (0 for control, 1 for ALS/PDC)"""
    if pd.isna(value):
        return None
    
    # Extract the value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Classify based on clinical phenotypes
    if value.lower() in ['healthy control', 'ct1', 'ct2', 'ct3']:
        return 0  # Control
    elif any(term in value.lower() for term in ['als', 'pdc', 'amyotrophic lateral sclerosis', 'parkinsonism-dementia complex']):
        return 1  # ALS/PDC case
    elif value.lower() in ['alzheimer`s disease', 'ad1', 'ad2', 'ad3']:
        return 0  # Treat Alzheimer's as non-ALS/PDC for this analysis
    else:
        return None  # Unknown

def convert_age(value):
    """Convert age value to continuous numeric"""
    if pd.isna(value):
        return None
    
    # Extract the value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to convert to integer
    try:
        return int(value)
    except ValueError:
        return None  # Return None if conversion fails

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    if pd.isna(value):
        return None
    
    # Extract the value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    else:
        return None  # Unknown

# 3. Save Metadata
# Determine trait availability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Check if trait_row is not None
if trait_row is not None:
    # Create a proper DataFrame structure for the geo_select_clinical_features function
    # In GEO data format, rows are characteristics and columns are samples
    # Here we'll create a simple DataFrame where each row corresponds to a sample characteristic key
    
    # Sample characteristics dictionary
    sample_char_dict = {
        0: ['subject id: CT1', 'subject id: CT2', 'subject id: CT3', 'subject id: AD1', 'subject id: AD2', 'subject id: AD3', 'clinical phenotypes: ALS', 'clinical phenotypes: ALS+D', 'clinical phenotypes: PDC+A', 'clinical phenotypes: PDC'], 
        1: ['clinical phenotypes: Healthy Control', 'clinical phenotypes: Alzheimer`s Disease', 'gender: Female', 'gender: Male'], 
        2: ['gender: Male', 'age: 66', 'age: 77', 'age: 70', 'age: 74', 'age: 76', 'age: 60', 'age: 79', 'age: 71', 'age: 63', 'age: 65', 'age: 81', 'age: 73', 'age: 72', 'age: 75', 'age: 85'], 
        3: ['age: 75', 'age: 76', 'age: 83', 'age: 84', 'age: 87', 'age: 88', 'age: 67', 'age: 68', 'age: 86', 'age: 74', 'tissue: Human Postmortem Brain'], 
        4: ['tissue: Human Postmortem Brain', 'tissue subtype: Frontal lobe', 'tissue subtype: Temporal lobe'], 
        5: ['tissue subtype: Frontal lobe', 'tissue subtype: Temporal lobe', float('nan')]
    }
    
    # For the geo_select_clinical_features function, we need a DataFrame with row indices as row numbers
    # and each value as a separate "sample"
    # Create a DataFrame with a single row for each key in the dictionary
    rows = []
    for key, values in sample_char_dict.items():
        row_data = {f'Sample_{i}': val for i, val in enumerate(values)}
        rows.append(row_data)
    
    # Convert to DataFrame
    clinical_data = pd.DataFrame(rows)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:", preview)
    
    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features: {'Sample_0': [0.0, nan, nan], 'Sample_1': [0.0, 66.0, nan], 'Sample_2': [0.0, 77.0, 0.0], 'Sample_3': [0.0, 70.0, 1.0], 'Sample_4': [0.0, 74.0, nan], 'Sample_5': [0.0, 76.0, nan], 'Sample_6': [1.0, 60.0, nan], 'Sample_7': [1.0, 79.0, nan], 'Sample_8': [1.0, 71.0, nan], 'Sample_9': [1.0, 63.0, nan], 'Sample_10': [nan, 65.0, nan], 'Sample_11': [nan, 81.0, nan], 'Sample_12': [nan, 73.0, nan], 'Sample_13': [nan, 72.0, nan], 'Sample_14': [nan, 75.0, nan], 'Sample_15': [nan, 85.0, nan]}
Clinical data saved to ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/clinical_data/GSE139384.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651254', 'ILMN_1651262', 'ILMN_1651315', 'ILMN_1651354',
       'ILMN_1651385', 'ILMN_1651405', 'ILMN_1651429', 'ILMN_1651438',
       'ILMN_1651498', 'ILMN_1651680', 'ILMN_1651705', 'ILMN_1651719',
       'ILMN_1651735', 'ILMN_1651745', 'ILMN_1651799', 'ILMN_1651819'],
      dtype='object', name='ID')

Gene data dimensions: 7154 genes × 33 samples


### Step 4: Gene Identifier Review

In [5]:
# These identifiers like "ILMN_1343291" are Illumina probe IDs, not standard human gene symbols
# They are from Illumina BeadArray microarray platforms and need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in the gene annotation dataframe correspond to probe IDs and gene symbols
# Looking at the gene annotation preview, we can see:
# - 'ID' column contains the ILMN_xxxx identifiers that match our gene expression data
# - 'Symbol' column contains the gene symbols we need to map to

# 2. Extract gene mapping data using the function from the library
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')

# Print preview of the mapping to verify
print("\nGene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression
# Use the apply_gene_mapping function which handles the many-to-many relation
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print information about the mapped gene data
print(f"\nAfter mapping: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print("\nFirst 10 mapped gene symbols:")
print(gene_data.index[:10])

# Save the gene expression data to a file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")



Gene mapping preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Gene': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB']}

After mapping: 5550 genes × 33 samples

First 10 mapped gene symbols:
Index(['A2BP1', 'A2M', 'AADACL1', 'AADAT', 'AAGAB', 'AARS', 'AARSD1',
       'AASDHPPT', 'AATK', 'ABAT'],
      dtype='object', name='Gene')

Gene expression data saved to ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/gene_data/GSE139384.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the index of gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 gene symbols after normalization: {normalized_gene_data.index[:5]}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the previously saved clinical data
try:
    # Load the clinical data that was already processed and saved in step 2
    clinical_data_df = pd.read_csv(out_clinical_data_file)
    
    # Transpose the data so samples are rows and features are columns
    clinical_data_df = clinical_data_df.T
    
    # Set proper column names, assuming first row is trait, second is Age, third is Gender
    clinical_data_df.columns = [trait, 'Age', 'Gender']
    
    # Remove the header row if it became the first row after transposition
    if clinical_data_df.index[0] == '0':
        clinical_data_df = clinical_data_df.iloc[1:]
    
    # Convert sample IDs to match gene expression data format
    # Remove 'Sample_' prefix if present
    clinical_data_df.index = clinical_data_df.index.str.replace('Sample_', 'GSM', regex=False)
    
    is_trait_available = True
    print(f"Loaded and restructured clinical data from {out_clinical_data_file}")
    print(f"Clinical data shape: {clinical_data_df.shape}")
    print(f"Clinical data preview: {preview_df(clinical_data_df, n=3)}")
except Exception as e:
    print(f"Error loading or processing clinical data: {e}")
    is_trait_available = False

# 3. Link clinical and genetic data if trait data is available
if is_trait_available:
    # Find common samples between clinical and gene data
    common_samples = set(clinical_data_df.index).intersection(set(normalized_gene_data.columns))
    if len(common_samples) == 0:
        print("No common samples found between clinical and gene expression data.")
        is_trait_available = False
        is_biased = True
        linked_data = pd.DataFrame()
    else:
        # Filter to keep only common samples
        clinical_data_filtered = clinical_data_df.loc[list(common_samples)]
        gene_data_filtered = normalized_gene_data[list(common_samples)]
        
        # Combine clinical and gene expression data
        linked_data = pd.concat([
            clinical_data_filtered,
            gene_data_filtered.T
        ], axis=1)
        
        print(f"Linked data shape before handling missing values: {linked_data.shape}")
        
        # Handle missing values
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
        
        # Determine if trait and demographic features are biased
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
else:
    print("Trait data is unavailable in this dataset.")
    is_biased = True  # Dataset can't be used without trait data
    linked_data = pd.DataFrame()  # Empty DataFrame

# 5. Validate and save cohort info
note = ""
if is_trait_available:
    note = "Dataset contains gene expression data from ALS patients with clinical information."
else:
    note = "Dataset contains gene expression data from ALS patients, but lacks trait information (disease status) required for associational studies."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data if not linked_data.empty else pd.DataFrame(index=normalized_gene_data.columns),
    note=note
)

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associational studies.")

Gene data shape after normalization: (5434, 33)
First 5 gene symbols after normalization: Index(['A2M', 'AADAT', 'AAGAB', 'AARS1', 'AARSD1'], dtype='object', name='Gene')


Normalized gene data saved to ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/gene_data/GSE139384.csv
Loaded and restructured clinical data from ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/clinical_data/GSE139384.csv
Clinical data shape: (16, 3)
Clinical data preview: {'Amyotrophic_Lateral_Sclerosis': [0.0, 0.0, 0.0], 'Age': [nan, 66.0, 77.0], 'Gender': [nan, nan, 0.0]}
No common samples found between clinical and gene expression data.
Abnormality detected in the cohort: GSE139384. Preprocessing failed.
Dataset deemed not usable for associational studies.
